In [1]:
from langchain.tools import tool
from langchain_groq import ChatGroq
import requests,os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Tool to call conversion rate API
@tool
def conversion_rate_api(base_currency:str,target_currency:str) -> float:
    'This tool will get latest conversion between two currencies'
    # API endpoint URL
    key = os.getenv('EXCHANGERATE_API_KEY')
    url = f'https://v6.exchangerate-api.com/v6/{key}/pair/{base_currency}/{target_currency}'
    data = requests.get(url)
    return data.json()['conversion_rate']

In [4]:
# Tool to multiply currency rate and amount
from langchain_core.tools import InjectedToolArg # To say we are going to injest this at run time
from typing import Annotated

@tool
def multiply_currency_rate_and_amount(currency_rate:float, amount:Annotated[float,InjectedToolArg]) -> float:
    'This tool will multiply the currency rate and amount'
    return currency_rate * amount

In [5]:
conversion_rate_api.invoke({'base_currency':'USD','target_currency':'INR'})

84.5138

In [6]:
multiply_currency_rate_and_amount.invoke({'currency_rate':84.5138,'amount':10})

845.138

In [7]:
llm = ChatGroq(model = 'Llama3-8b-8192')

In [9]:
#binding tools with llm
llm_with_tools = llm.bind_tools([conversion_rate_api,multiply_currency_rate_and_amount])

In [10]:
from langchain_core.messages import HumanMessage

In [13]:
question = "What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd"

In [24]:
history = []
history.append(HumanMessage(question))
response1 = llm_with_tools.invoke(history)
print(response1)

content='' additional_kwargs={'tool_calls': [{'id': 'call_y3jy', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'conversion_rate_api'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 1056, 'total_tokens': 1132, 'completion_time': 0.063333333, 'prompt_time': 0.146981218, 'queue_time': 0.08866352799999999, 'total_time': 0.210314551}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--cac1f99d-773c-41fe-acbe-5a389807d644-0' tool_calls=[{'name': 'conversion_rate_api', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_y3jy', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1056, 'output_tokens': 76, 'total_tokens': 1132}


In [25]:
history.append(response1)
response1.tool_calls # Here we are using Llama3 using Groq, if we use better model, it will identify two tools here

[{'name': 'conversion_rate_api',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_y3jy',
  'type': 'tool_call'}]

In [26]:
for tol in response1.tool_calls:
    # print(tol)
    if tol['name'] == 'conversion_rate_api':
        conversion_rate = conversion_rate_api.invoke(tol)
        history.append(conversion_rate)
    if tol['name'] == 'multiply_currency_rate_and_amount':
        tol['args']['conversion_rate'] = conversion_rate
        multiply_currency_rate_and_amount = multiply_currency_rate_and_amount.invoke(tol)
        history.append(multiply_currency_rate_and_amount)
        

In [27]:
history

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_y3jy', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'conversion_rate_api'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 1056, 'total_tokens': 1132, 'completion_time': 0.063333333, 'prompt_time': 0.146981218, 'queue_time': 0.08866352799999999, 'total_time': 0.210314551}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cac1f99d-773c-41fe-acbe-5a389807d644-0', tool_calls=[{'name': 'conversion_rate_api', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_y3jy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1056, 'output_tokens': 76, 'total_tokens': 11

In [29]:
response2 = llm_with_tools.invoke(history)
print(response2)

content='' additional_kwargs={'tool_calls': [{'id': 'call_wx8m', 'function': {'arguments': '{"currency_rate":0.01183,"amount":10}', 'name': 'multiply_currency_rate_and_amount'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 1133, 'total_tokens': 1177, 'completion_time': 0.036666667, 'prompt_time': 0.140792687, 'queue_time': 0.07349695, 'total_time': 0.177459354}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--6a8d3ffa-869b-4d61-a737-cfd745365691-0' tool_calls=[{'name': 'multiply_currency_rate_and_amount', 'args': {'currency_rate': 0.01183, 'amount': 10}, 'id': 'call_wx8m', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1133, 'output_tokens': 44, 'total_tokens': 1177}


In [30]:
response2.tool_calls

[{'name': 'multiply_currency_rate_and_amount',
  'args': {'currency_rate': 0.01183, 'amount': 10},
  'id': 'call_wx8m',
  'type': 'tool_call'}]

In [31]:
for tol in response2.tool_calls:
    # print(tol)
    if tol['name'] == 'conversion_rate_api':
        conversion_rate = conversion_rate_api.invoke(tol)
        history.append(conversion_rate)
    if tol['name'] == 'multiply_currency_rate_and_amount':
        tol['args']['conversion_rate'] = conversion_rate
        multiply_currency_rate_and_amount = multiply_currency_rate_and_amount.invoke(tol)
        history.append(multiply_currency_rate_and_amount)

In [32]:
history

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_y3jy', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'conversion_rate_api'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 1056, 'total_tokens': 1132, 'completion_time': 0.063333333, 'prompt_time': 0.146981218, 'queue_time': 0.08866352799999999, 'total_time': 0.210314551}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cac1f99d-773c-41fe-acbe-5a389807d644-0', tool_calls=[{'name': 'conversion_rate_api', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_y3jy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1056, 'output_tokens': 76, 'total_tokens': 11

In [33]:
response3 = llm_with_tools.invoke(history)
print(response3)

content='' additional_kwargs={'tool_calls': [{'id': 'call_845j', 'function': {'arguments': '{"currency_rate":0.1183,"amount":10}', 'name': 'multiply_currency_rate_and_amount'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 1166, 'total_tokens': 1214, 'completion_time': 0.04, 'prompt_time': 0.146595597, 'queue_time': 0.09046796800000001, 'total_time': 0.186595597}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--453eb11b-e66d-4e6d-b81c-f099ab46ae5c-0' tool_calls=[{'name': 'multiply_currency_rate_and_amount', 'args': {'currency_rate': 0.1183, 'amount': 10}, 'id': 'call_845j', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1166, 'output_tokens': 48, 'total_tokens': 1214}


In [ ]:
# If we use Good LLM, All tools will be identified ata first and If we pass all human,AI,tools messages, We get final output.